In [2]:
import puzzle as puzzle
import tableau as tableau
import matplotlib.pyplot as plt
import matplotlib.patches as pat

In [3]:
def convertStringToDiagram(string):
    L = list(string)
    n = len(L)
    k = 0
    I = []
    for (i, s) in enumerate(L):
        if s == "1":
            k += 1
            I.append(i+1)
    
    diagram = [n - k - I[0] + 1]
    for i in range(k-1):
        diagram.append(diagram[i] - (I[i+1] - I[i]) + 1)
    return diagram


def convertDiagramToString(inputDiagram, n, k):
    diagram = inputDiagram.copy()
    I = [n - k + 1 - diagram[0]]
    if len(diagram) < k:
        diagram += [0 for i in range(k-len(diagram))]
    for j in range(k-1):
        I.append(I[j] + diagram[j] - diagram[j+1] + 1)
    string = [0 for j in range(n)]
    for s in I:
        string[s-1] = 1
    result = list(map( lambda x: str(x), string ))
    return "".join(result)

def min_lexicographical(x, y):
    if x[0] < y[0]:
        return x
    if x[0] > y[0]:
        return y
    if x[1] < y[1]:
        return x
    if x[1] > y[1]:
        return y
    return x

def mysort(ary):
    l = len(ary)
    if l <= 1:
        return ary
    
    former = mysort(ary[:int(l/2)].copy())
    latter = mysort(ary[int(l/2):].copy())
    result = []
    i = 0
    j = 0
    while (i < len(former)) and (j < len(latter)):
        x = former[i]
        y = latter[j]
        if min_lexicographical(x, y) == x:
            result.append(x)
            i += 1
        else:
            result.append(y)
            j += 1
    result += former[i:].copy()
    result += latter[j:].copy()
    return result


In [4]:
l = [3,3,3]
m = [3,2,2]
n = [5,4,3]
c = 8
k = 3


#puzzleList = puzzle.calculateAllPuzzle(convertDiagramToString(l, c, k), convertDiagramToString(m, c, k), convertDiagramToString(n, c, k))
tableauList = tableau.calculateAllEdgeLabeledTableaux(l, m, n, c-k)

#puzzleWeights = [p.weight(mode="int") for p in puzzleList]
#tableauWeights = [t.weight(l, c-k, mode="int") for t in tableauList]

#orderedTableauList = [0 for i in range(len(tableauList))]
#for t in tableauList:
#    for i, p in enumerate(puzzleList):
#        if set(tuple( mysort(t.weight(l, c-k, mode="int")) )) == set(tuple( mysort(p.weight(mode="int")) )):
#            orderedTableauList[i] = t

In [4]:
tableauTrace = [tableauList[-1]]
#puzzleTrace = [puzzleList[-1]]
largeShape = n
smallShape = l

while True:
    t = tableauTrace[-1].copy()
    b = t.elements[0][smallShape[0]-1]
    while not b.isInnerCorner():
        b = b.lowerBox
    smallShape[b.rowNumber()] -= 1
    t.equivariantConsecutiveSlide(b)
    largeShape = t.diagram

    #puzzleList = puzzle.calculateAllPuzzle(convertDiagramToString(smallShape, c, k), convertDiagramToString(m, c, k), convertDiagramToString(largeShape, c, k))
    #puzzleTrace.append(puzzleList[0].copy())
    tableauTrace.append(t.copy())
    if t.isSuperStandard():
        break


In [5]:
for t in tableauTrace:
    print(t)
    print(t.outputEdgeLabels())
    print("")

[0, 0, 0, 0, 5]
[0, 0, 0, 0, 7]
[0, 2, 6],
[set(),set(),set(),set(),set()]
[set(),set(),{3},{4},set()]
[{1},set(),set(),]

[0, 0, 0, 0, 5]
[0, 0, 0, 4, 7]
[0, 2, 6],
[set(),set(),set(),set(),set()]
[set(),set(),{3},set(),set()]
[{1},set(),set(),]

[0, 0, 0, 4, 5]
[0, 0, 0, 7]
[0, 2, 6],
[set(),set(),set(),set(),set()]
[set(),set(),{3},set(),]
[{1},set(),set(),]

[0, 0, 0, 4, 5]
[0, 0, 3, 7]
[0, 2, 6],
[set(),set(),set(),set(),set()]
[set(),set(),set(),set(),]
[{1},set(),set(),]

[0, 0, 3, 4, 5]
[0, 0, 6, 7]
[0, 2],
[set(),set(),set(),set(),set()]
[set(),set(),set(),set(),]
[{1},set(),]

[0, 0, 3, 4, 5]
[0, 2, 6, 7]
[0],
[set(),set(),set(),set(),set()]
[set(),set(),set(),set(),]
[{1},]

[0, 2, 3, 4, 5]
[0, 6, 7]
[0],
[set(),set(),set(),set(),set()]
[set(),set(),set(),]
[{1},]

[0, 2, 3, 4, 5]
[0, 6, 7]
[1],
[set(),set(),set(),set(),set()]
[set(),set(),set(),]
[set(),]

[0, 2, 3, 4, 5]
[1, 6, 7],
[set(),set(),set(),set(),set()]
[set(),set(),set(),]

[1, 2, 3, 4, 5]
[6, 7],
[set(),set(),s